# Quantum State Iterative Projection via Optimized Projectors

## Overview
This notebook demonstrates a computational method to iteratively transform an initial quantum state. The process involves:
1. Generating an initial N-qubit quantum state, `|psi_initial>`, using a `RealAmplitudes` ansatz. This state serves as our simulated "unknown" state.
2. Defining a parameterized two-qubit unitary, `U(theta)`, using a `TwoLocal` circuit.
3. For each pair of qubits `(q1, q2)` (including nearest neighbors and optionally a wrap-around pair), constructing a parameterized rank-1 projector `P_q1q2(theta) = U(theta) (|00><00|_q1q2) U_dag(theta)`.
4. Optimizing the parameters `theta` for each `P_q1q2(theta)` to minimize its expectation value with respect to `|psi_initial>`, i.e., find `P_q1q2*` such that `<psi_initial|P_q1q2*|psi_initial>` is close to zero. This optimization is performed using SciPy's COBYLA algorithm.
5. Sequentially applying a "filter" operator `K_q1q2 = (I - P_q1q2*)` to the evolving state and renormalizing after each application.
6. Analyzing the final state `|psi_final>` by comparing it to `|psi_initial>` (e.g., via fidelity) and checking expectation values.

The goal is to observe how the initial state transforms under these tailored projections and to what extent it retains its original characteristics, potentially offering a form of state characterization if the fidelity between initial and final states is high.

## Requirements
This notebook requires the following Python libraries:
- `numpy`: For numerical operations.
- `qiskit`: For quantum circuit simulation and quantum information tools.
- `scipy`: For classical optimization algorithms (COBYLA).

## 1. Imports and Setup

In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import RealAmplitudes, TwoLocal
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp, state_fidelity

# Import SciPy's optimizer
try:
    from scipy.optimize import minimize
except ModuleNotFoundError:
    print("ERROR: scipy is not installed. Please install it: pip install scipy")
    # Define a dummy minimize to allow the rest of the notebook to be parsed
    # if scipy is missing, though it will fail at runtime.
    def minimize(*args, **kwargs):
        raise RuntimeError("scipy.optimize.minimize not found. Please install scipy.")

## 2. Configuration Parameters
Define the main parameters for the simulation, such as the number of qubits, ansatz depths, optimizer settings, and the random seed for reproducibility.

In [ ]:
# --- Configuration ---
NUM_QUBITS = 5
PSI_ANSATZ_DEPTH = 5       # Depth of RealAmplitudes for psi_initial
PROJECTOR_U_REPS = 1       # Repetitions in TwoLocal for U(theta)
SCIPY_COBYLA_MAXITER = 10000 # Max iterations for COBYLA optimizer
SCIPY_COBYLA_TOL = 1e-7      # Tolerance for COBYLA optimizer
SEED = 43                    # Seed for NumPy random number generation

# Set NumPy's random seed for reproducibility of NumPy operations
np.random.seed(SEED)
print(f"NumPy random seed set to: {SEED}")

print(f"--- Quantum State Iterative Projection (Using SciPy COBYLA) ---")
print(f"Configuration: Qubits={NUM_QUBITS}, Psi Depth={PSI_ANSATZ_DEPTH}")
print(f"Optimizer: SciPy COBYLA, Maxiter={SCIPY_COBYLA_MAXITER}, Tol={SCIPY_COBYLA_TOL}\n")

## 3. Step 1: Create Initial Quantum State `|psi_initial>`
An N-qubit initial state is generated using the `RealAmplitudes` ansatz with random parameters. This state will be the subject of our iterative projection process.

In [ ]:
print("Step 1: Creating initial state |psi_initial>...")
psi_ansatz = RealAmplitudes(NUM_QUBITS, reps=PSI_ANSATZ_DEPTH, entanglement='linear')
num_psi_params = psi_ansatz.num_parameters
# These random parameters are reproducible due to np.random.seed(SEED)
random_psi_params = np.random.rand(num_psi_params) * 2 * np.pi
psi_circuit = psi_ansatz.assign_parameters(random_psi_params)

try:
    psi_initial_vector = Statevector(psi_circuit)
    print(f"|psi_initial> statevector created with {num_psi_params} parameters.")
    # print(psi_initial_vector) # Optional: view the statevector
except Exception as e:
    print(f"CRITICAL ERROR: Could not create Statevector. Qiskit installation is likely broken: {e}")
    import sys
    sys.exit(1) # Stop execution if basic Qiskit functionality fails

## 4. Step 2: Define Parameterized Projectors `P_ij(theta)`

### 4.1 Define Static Base Projectors
We define a static base rank-1 projector $P_{base} = |00\rangle\langle00|_{q1q2} \otimes I_{\text{others}}$ for each qubit pair `(q1, q2)`. This projector acts as $|00\rangle\langle00|$ on the specified pair and identity on all other qubits. It's represented as a `SparsePauliOp`.
The Pauli string representation for $|00\rangle\langle00|_{q1q2}$ is $0.25(I+Z_{q1})(I+Z_{q2}) = 0.25(I_{q1}I_{q2} + I_{q1}Z_{q2} + Z_{q1}I_{q2} + Z_{q1}Z_{q2})$.
Note on Pauli string convention for `SparsePauliOp.from_list`: Qiskit typically interprets strings such that the rightmost character corresponds to qubit 0. The code includes a `[::-1]` reversal to align the constructed strings with this convention, assuming the string `"IZZII"` (for 5 qubits) means Z on qubit 2 if the string is directly used, or Z on qubit `NUM_QUBITS-1-2` if reversed.

In [ ]:
proj_00_on_qubits_static_pauli_sum = {}
# This loop defines projectors for nearest-neighbor pairs only
# If wrap-around or other pairs are needed, qubit_pairs_indices in Step 3
# would need to be adjusted, and this dictionary populated accordingly.
for q1_local_idx in range(NUM_QUBITS - 1):
    q2_local_idx = q1_local_idx + 1
    pauli_list_for_proj = [
        ("I" * NUM_QUBITS, 0.25),
        ("".join(['Z' if i == q1_local_idx else 'I' for i in range(NUM_QUBITS)]), 0.25),
        ("".join(['Z' if i == q2_local_idx else 'I' for i in range(NUM_QUBITS)]), 0.25),
        ("".join(['Z' if i == q1_local_idx or i == q2_local_idx else 'I' for i in range(NUM_QUBITS)]), 0.25)
    ]
    # Reverse strings to match Qiskit's typical convention for from_list 
    # (rightmost character is qubit 0)
    reversed_pauli_list = [(p_str[::-1], coeff) for p_str, coeff in pauli_list_for_proj]
    proj_00_on_qubits_static_pauli_sum[(q1_local_idx, q2_local_idx)] = SparsePauliOp.from_list(reversed_pauli_list)

### 4.2 Define Unitary Ansatz `U(theta)`
A `TwoLocal` circuit is used as the ansatz for the 2-qubit unitary `U(theta)`. This unitary will rotate the base projector: $P_{q1q2}(\theta) = U(\theta) P_{base} U^\dagger(\theta)$.

In [ ]:
u_ansatz_2q = TwoLocal(2, rotation_blocks=['ry', 'rz'], entanglement_blocks='cx', 
                       reps=PROJECTOR_U_REPS, entanglement='linear')
num_u_params = u_ansatz_2q.num_parameters
print(f"\nStep 2: Parametrized projectors P_ij(theta) structure defined.")
print(f"U(theta) for projectors is TwoLocal with {num_u_params} parameters (reps={PROJECTOR_U_REPS}).")

## 5. Step 3: Optimize Projector Parameters `P_ij*`
For each selected qubit pair `(q1, q2)`, the parameters `theta` of `U(theta)` are optimized to minimize the expectation value `<psi_initial|P_q1q2(theta)|psi_initial>`. SciPy's COBYLA algorithm is used for this optimization. The optimized projectors $P_{q1q2}^*$ (as `Operator` objects) are stored.

In [ ]:
print("\nStep 3: Optimizing projector parameters P_ij* against |psi_initial>...")
# Defines which pairs of qubits the projectors will act on.
# Currently, only nearest neighbors.
qubit_pairs_indices = [(i, i + 1) for i in range(NUM_QUBITS - 1)]
optimized_projectors_P_star = [] # Stores {'qubits':(q1,q2), 'P_star_op': Operator}

for q_pair_idx, (q1, q2) in enumerate(qubit_pairs_indices):
    print(f"  Optimizing P_{q1}{q2}(theta) acting on qubits ({q1}, {q2})...")
    
    # Retrieve the static base projector for the current pair
    proj_00_full_static_spo = proj_00_on_qubits_static_pauli_sum[(q1, q2)]

    # Cost function to be minimized by SciPy
    def cost_function_projector(params_theta):
        try:
            # Assign current parameters to the 2-qubit unitary ansatz
            u_circuit_2q = u_ansatz_2q.assign_parameters(params_theta)
            # Embed the 2-qubit U(theta) into the full N-qubit system
            u_full_qc = QuantumCircuit(NUM_QUBITS)
            u_full_qc.compose(u_circuit_2q, qubits=[q1, q2], inplace=True)
            u_full_op = Operator(u_full_qc)
            
            # Form the full projector P_ij(theta) = U @ P_base @ U_dag
            p_ij_full_op_object = u_full_op @ proj_00_full_static_spo @ u_full_op.adjoint()
            
            # Calculate expectation value with the initial state
            expectation_value = np.real(psi_initial_vector.expectation_value(p_ij_full_op_object))
            return expectation_value
        except Exception as e_cost:
            # Handle errors during cost function evaluation (e.g., Qiskit issues)
            print(f"ERROR in cost_function_projector for P_{q1}{q2} with params {params_theta}: {e_cost}.")
            return np.nan # Return NaN to signal failure to the optimizer

    # Initial guess for U(theta) parameters (reproducible due to np.random.seed)
    initial_u_params = np.random.rand(num_u_params) * 2 * np.pi
    
    try:
        # Perform optimization using SciPy's COBYLA
        opt_result = minimize(fun=cost_function_projector,
                              x0=initial_u_params,
                              method='COBYLA',
                              tol=SCIPY_COBYLA_TOL,
                              options={'maxiter': SCIPY_COBYLA_MAXITER, 'disp': False})
        
        optimized_u_params = opt_result.x
        min_expectation_value = opt_result.fun
        nfev = opt_result.nfev
        success = opt_result.success

        print(f"    P_{q1}{q2}*: Optimized <psi_init|P*|psi_init> = {min_expectation_value:.6e}, Evals = {nfev}, Success: {success}")

        if not success and not np.isnan(min_expectation_value):
             print(f"    Warning: SciPy COBYLA optimization for P_{q1}{q2}* did not converge successfully.")
        elif np.isnan(min_expectation_value):
            print(f"    ERROR: Optimization for P_{q1}{q2}* failed (NaN result). Skipping this projector.")
            continue # Skip to the next projector pair

        # Reconstruct the optimized P_ij* Operator object
        u_circuit_2q_opt = u_ansatz_2q.assign_parameters(optimized_u_params)
        u_full_qc_opt = QuantumCircuit(NUM_QUBITS)
        u_full_qc_opt.compose(u_circuit_2q_opt, qubits=[q1, q2], inplace=True)
        u_full_op_opt = Operator(u_full_qc_opt)
        
        p_ij_star_op_object = u_full_op_opt @ proj_00_full_static_spo @ u_full_op_opt.adjoint()
        optimized_projectors_P_star.append({'qubits': (q1,q2), 'P_star_op': p_ij_star_op_object})

    except RuntimeError as e_scipy: # Catch errors specifically from SciPy's minimize
        print(f"    ERROR: SciPy COBYLA optimization failed for P_{q1}{q2}*: {e_scipy}")
        continue
    except Exception as e_general_opt: # Catch any other errors during this iteration
        print(f"    ERROR: General error during optimization or reconstruction for P_{q1}{q2}*: {e_general_opt}")
        continue

## 6. Step 4: Iteratively Apply `(I - P_ij*)` and Renormalize
The state `|psi_initial>` is iteratively transformed. In each step, for a given optimized projector $P_{q1q2}^*$, the operator $K = (I - P_{q1q2}^*)$ is applied to the current state. The resulting state is then renormalized. This process is repeated for all optimized projectors in a defined order.

In [ ]:
print("\nStep 4: Iteratively applying (I - P_ij*) projection and renormalizing...")

if not optimized_projectors_P_star:
    print("Warning: No optimized projectors P_ij* were successfully created. Cannot perform iterative projection.")
    final_psi_vector = psi_initial_vector.copy() # No projection applied
else:
    current_psi_vector = psi_initial_vector.copy() # Start with a copy
    # N-qubit Identity Operator
    identity_op = Operator(np.eye(2**NUM_QUBITS, dtype=complex))

    # Define the order in which projectors are applied (matches optimization order)
    projection_order = [(i, i + 1) for i in range(NUM_QUBITS - 1)] 

    for q_idx, (q_target1, q_target2) in enumerate(projection_order):
        # Find the corresponding optimized P_star for this pair
        p_star_info = next((p_info for p_info in optimized_projectors_P_star if p_info['qubits'] == (q_target1, q_target2)), None)

        if p_star_info is None:
            print(f"  Warning: No optimized P_star found for qubits ({q_target1},{q_target2}). Skipping this projection step.")
            continue
        
        p_ij_star_op = p_star_info['P_star_op']
        print(f"  Applying (I - P_{q_target1}{q_target2}*) to current state...")

        # Define K_ij = I - P_ij*
        k_ij_op = identity_op - p_ij_star_op
        
        # Apply K_ij to the current state vector
        projected_psi_unnormalized = current_psi_vector.evolve(k_ij_op)
        
        # Robust norm calculation using the underlying .data array
        vec_data = projected_psi_unnormalized.data
        norm_val = np.linalg.norm(vec_data)

        if norm_val < 1e-9: # Threshold for norm being effectively zero
            print(f"    WARNING: State norm after applying (I - P_{q_target1}{q_target2}*) is near zero ({norm_val:.2e}).")
            print(f"    This implies the state was almost entirely in the subspace projected onto by P_{q_target1}{q_target2}*.")
            print(f"    Cannot renormalize meaningfully. Stopping iterative projection.")
            # Create a new Statevector from the (potentially zero) data
            current_psi_vector = Statevector(projected_psi_unnormalized.data, dims=projected_psi_unnormalized.dims())
            break # Exit the loop for projections
        
        # Explicitly create a new Statevector from the normalized data array
        # This ensures internal Qiskit Statevector validity.
        normalized_data_arr = projected_psi_unnormalized.data / norm_val
        current_psi_vector = Statevector(normalized_data_arr, dims=projected_psi_unnormalized.dims())
        
        # Verify norm of the new Statevector object
        new_norm_check = np.linalg.norm(current_psi_vector.data)
        print(f"    State renormalized. New norm after explicit creation: {new_norm_check:.7f}")

    final_psi_vector = current_psi_vector

## 7. Step 5: Analyze the Final State
The final state `|psi_final>` is analyzed. Key metrics include:
- A sample of its amplitudes.
- Fidelity between `|psi_initial>` and `|psi_final>`.
- Expectation values of the optimized projectors $P_{q1q2}^*$ with respect to both `|psi_initial>` (should be small) and `|psi_final>`.

In [ ]:
print("\nStep 5: Final Projected State Analysis...")

# Explicitly re-normalize final_psi_vector before use, just in case of tiny accumulated errors
# Only if its norm is not already effectively zero.
final_norm_val = np.linalg.norm(final_psi_vector.data)
if final_norm_val > 1e-9: # If it's not a zero vector
    # Only re-normalize if the norm is significantly different from 1.0
    if abs(final_norm_val - 1.0) > 1e-7:
        print(f"  Normalizing final_psi_vector before analysis (current norm: {final_norm_val:.7f})")
        final_psi_vector = Statevector(final_psi_vector.data / final_norm_val, dims=final_psi_vector.dims())
        print(f"  Norm of final_psi_vector after re-check: {np.linalg.norm(final_psi_vector.data):.7f}")
else:
    print(f"  Final state is a near-zero vector (norm: {final_norm_val:.2e}). Analysis might be limited.")

print("Initial state |psi_initial> (first 4 amplitudes):")
print(psi_initial_vector.data[:4])
print("\nFinal state |psi_final> after iterative projections (first 4 amplitudes):")
print(final_psi_vector.data[:4]) 

try:
    # Check if final_psi_vector is valid for fidelity calculation
    if final_norm_val < 1e-9: # If it's essentially a zero vector
        print("\nFidelity calculation skipped as final state is a zero vector.")
        fidelity_final_vs_initial = 0.0
    else:
        # Ensure psi_initial_vector is also perfectly normalized if it's used elsewhere
        if abs(np.linalg.norm(psi_initial_vector.data) - 1.0) > 1e-9:
             print("Warning: Initial state vector norm is not 1. Re-normalizing for fidelity.")
             psi_initial_vector = Statevector(psi_initial_vector.data / np.linalg.norm(psi_initial_vector.data))

        fidelity_final_vs_initial = state_fidelity(psi_initial_vector, final_psi_vector)
        print(f"\nFidelity(|psi_initial>, |psi_final>): {fidelity_final_vs_initial:.8f}")

    print("\nExpectation values of P_ij* with respect to |psi_initial> (should be small):")
    for p_info in optimized_projectors_P_star:
        q1,q2 = p_info['qubits']
        exp_val = np.real(psi_initial_vector.expectation_value(p_info['P_star_op']))
        print(f"  <psi_initial| P_{q1}{q2}* |psi_initial>: {exp_val:.6e}")

    print("\nExpectation values of P_ij* with respect to |psi_final> (may not be small):")
    for p_info in optimized_projectors_P_star:
        q1,q2 = p_info['qubits']
        if final_norm_val > 1e-9: # Only compute if final_psi_vector is not zero
            exp_val_final = np.real(final_psi_vector.expectation_value(p_info['P_star_op']))
            print(f"  <psi_final| P_{q1}{q2}* |psi_final>: {exp_val_final:.6e}")
        else:
            print(f"  <psi_final| P_{q1}{q2}* |psi_final>: Not computed (final state is near zero vector)")

except Exception as e_final_analysis:
    print(f"ERROR during final state analysis: {e_final_analysis}")
    # For debugging, print the norm of the final vector just before the error
    if 'final_psi_vector' in locals(): # Check if variable exists
        print(f"DEBUG: Norm of final_psi_vector right before error: {np.linalg.norm(final_psi_vector.data)}")
        # Check Qiskit's own validity check for the Statevector
        if hasattr(final_psi_vector, 'is_valid'):
            print(f"DEBUG: final_psi_vector.is_valid(atol=1e-9): {final_psi_vector.is_valid(atol=1e-9)}")
        else:
            print("DEBUG: final_psi_vector.is_valid() method not available (older Qiskit version?)")

print("\n--- Iterative Projection Script Finished ---")

## 8. Execution Guidance

1.  **Kernel:** Ensure you are running this notebook with a Python 3 kernel that has `qiskit`, `numpy`, and `scipy` installed.
2.  **Run Cells Sequentially:** Execute the cells in order from top to bottom.
3.  **Adjust Configuration:** Modify parameters in Cell 2 (Configuration Parameters) as needed:
    *   `NUM_QUBITS`: Number of qubits for the states.
    *   `PSI_ANSATZ_DEPTH`: Depth of the `RealAmplitudes` circuit for the initial state. Higher depth generally means more entanglement and complexity.
    *   `PROJECTOR_U_REPS`: Depth of the `TwoLocal` circuit for the unitary $U(\theta)$ within the projectors.
    *   `SCIPY_COBYLA_MAXITER`, `SCIPY_COBYLA_TOL`: Control the behavior of the COBYLA optimizer. Higher `maxiter` may lead to better optimization but takes longer. Lower `tol` seeks higher precision.
    *   `SEED`: Change the seed to generate different random initial states and initial optimizer parameters.
4.  **Wrap-around Projector (Optional):** The current code in Step 3 (`qubit_pairs_indices`) only considers nearest-neighbor projectors ($P_{01}, P_{12}, \dots, P_{N-2,N-1}$). If you want to include a projector acting on the last and first qubits ($P_{N-1,0}$), you would need to:
    *   Add the pair `(NUM_QUBITS - 1, 0)` to `qubit_pairs_indices` in Step 3.
    *   Ensure `proj_00_on_qubits_static_pauli_sum` in Step 2.1 is populated for this wrap-around pair.
    *   Adjust `projection_order` in Step 4 if you want a specific sequence including this wrap-around projector.

### Expected Output
- Print statements indicating the progress through each step.
- For each optimized projector $P_{ij}^*$, the minimized expectation value $\langle\psi_{initial}|P_{ij}^*|\psi_{initial}\rangle$, number of optimizer evaluations, and success status.
- During the iterative projection (Step 4), messages about renormalization and norms.
- In the final analysis (Step 5):
    - The first few amplitudes of the initial and final state vectors.
    - The fidelity $F(|\psi_{initial}\rangle, |\psi_{final}\rangle)$.
    - Expectation values of each $P_{ij}^*$ with respect to both the initial and final states.

If the fidelity between the initial and final states is high, it suggests that the initial state was already largely orthogonal to the subspaces defined by the optimized projectors $P_{ij}^*$. The set of optimized parameters $\theta^*$ for these projectors then provides a characterization of the initial state.